In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import pickle as pkl
import typing
from typing import List
from functools import partial

import torch
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import seaborn as sns
from tqdm import trange

from mtt.models.convolutional import load_model
from mtt.data import (
    vector_to_image,
    stack_images,
    rolling_window,
    VectorData,
    StackedImageData,
)
from mtt.peaks import find_peaks
from mtt.utils import compute_ospa_components, compute_ospa

rng = np.random.default_rng()
sns.set_theme(
    context="paper",
    style="whitegrid",
    rc={
        "figure.figsize": (3.5, 3.5),
        "figure.dpi": 150,
        "savefig.dpi": 1000,
        "figure.constrained_layout.use": True,
        "pdf.fonttype": 42,
    },
)

In [ ]:
scale = 1
simulation_idx = 40

with open(f"../data/test/{scale}km/simulations.pkl", "rb") as f:
    dataset_vectors: List[List[VectorData]] = pkl.load(f)

images = map(
    partial(vector_to_image, img_size=128 * scale), dataset_vectors[simulation_idx]
)
stacked = typing.cast(List[StackedImageData], rolling_window(stack_images(images)))

In [ ]:
# find all simulations where the sensor is within the simulation window
for i, simulation_vectors in enumerate(dataset_vectors):
    vectors = simulation_vectors[0]
    sensors_within_window = np.all(
        np.abs(vectors.sensor_positions)
        <= np.array([vectors.simulator.window_width / 2] * 2),
        axis=1,
    )
    if sensors_within_window.sum() > 2:
        print(i)

In [ ]:
cardinality = []
for simulation in dataset_vectors:
    cardinality.append([])
    for step in simulation:
        cardinality[-1].append(len(step.target_positions))

In [ ]:
simulator = dataset_vectors[0][0].simulator
window = simulator.window_width
extent = [-window / 2, window / 2, -window / 2, window / 2]
n_detections = (
    simulator.n_sensors * np.pi * simulator.sensors[0].range_max ** 2 / 1000**2
)

In [ ]:
# Make CNN Predictions
# model, name = load_model("../models/58c6fd8a.ckpt")
# model, name = load_model("wandb://damowerko/mtt/4uc51x21")
# model, name = load_model("wandb://damowerko/mtt/rmwq3ref")
# model, name = load_model("wandb://damowerko/mtt/eiytli6p")
# model, name = load_model("wandb://damowerko/mtt/e7ivqipk")
model, name = load_model("../models/e7ivqipk.ckpt")
model = model.cuda()
model.eval()

In [ ]:
filt_idx = -1
method = "gmm"
n_peaks_scale = 1.0

with torch.no_grad():
    x = torch.stack([d.sensor_images for d in stacked], dim=0)
    output_images = model(x.cuda())
    pred_img = output_images.clamp(min=0).cpu().numpy() * n_peaks_scale

n = pred_img.shape[0]
width = simulator.simulation_width
window = simulator.window_width
extent = [-window / 2, window / 2, -window / 2, window / 2]

predictions_cnn = []
for i in trange(n):
    predictions_cnn.append(
        find_peaks(pred_img[i][filt_idx], width=window, method=method).means
    )

cardinality_cnn = pred_img.sum(axis=(-1, -2)).mean().item()
print(f"Mean cardinality estimate: {cardinality_cnn:.2f}")
cardinality_truth = np.mean(
    [len(d.info[filt_idx]["target_positions"]) for d in stacked]
)
print(f"Mean cardinality truth: {cardinality_truth}")
print(f"Ratio {cardinality_truth / cardinality_cnn:.2f}")

In [ ]:
step_idx = 99

with sns.axes_style("ticks"):
    fig, axs = plt.subplots(1, 3, sharey=True, figsize=(7.0, 3.5))
    imshow_kwargs = dict(
        extent=extent,
        origin="lower",
        cmap="gray_r",
    )

    axs[0].set_title("Sensor Measurements")
    axs[0].imshow(stacked[step_idx].sensor_images[filt_idx].cpu(), **imshow_kwargs)
    axs[0].plot(
        *stacked[step_idx].info[filt_idx]["sensor_positions"].T, "ro", markersize=5
    )
    axs[0].set_xlim(xmin=extent[0], xmax=extent[1])
    axs[0].set_ylim(ymin=extent[2], ymax=extent[3])

    axs[1].set_title("Target Positions")
    axs[1].imshow(stacked[step_idx].target_images[filt_idx].cpu(), **imshow_kwargs)

    axs[2].set_title("CNN Output")
    axs[2].imshow(pred_img[step_idx][filt_idx], **imshow_kwargs)

    axs[0].set_ylabel("$y$ (m)")
    for ax in axs:
        ax.set_xlabel("$x$ (m)")

    plt.savefig("../figures/mtt_images.pdf", bbox_inches="tight")
    plt.show()

In [ ]:
with sns.axes_style("white"):
    figscale = 2.0
    fontsize = 22
    markersize = figscale * 5

    fig, axs = plt.subplots(1, 4, figsize=(figscale * 7.5, figscale * 3))
    for ax in axs:
        ax.set_aspect("equal")
        ax.set_xticks([])
        ax.set_yticks([])
        ax.set_xlim(extent[:2])
        ax.set_ylim(extent[2:])

    imshow_kwargs = dict(
        extent=extent,
        origin="lower",
        cmap="gray_r",
    )

    # Plot measurement positions
    measurements = np.concatenate(
        stacked[step_idx].info[filt_idx]["measurements"]
        + stacked[step_idx].info[filt_idx]["clutter"],
        axis=0,
    )

    axs[0].set_title("Sensor Measurements", fontsize=fontsize)
    axs[0].plot(*measurements.T, "b.", markersize=markersize)

    axs[1].set_title("Measurement Image", fontsize=fontsize)
    axs[1].imshow(stacked[step_idx].sensor_images[filt_idx].cpu(), **imshow_kwargs)
    axs[1].set_xlim(xmin=extent[0], xmax=extent[1])
    axs[1].set_ylim(ymin=extent[2], ymax=extent[3])

    axs[2].set_title("CNN Output", fontsize=fontsize)
    axs[2].imshow(pred_img[step_idx][filt_idx], **imshow_kwargs)

    axs[3].set_title("Position Estimates", fontsize=fontsize)
    axs[3].plot(*predictions_cnn[step_idx].T, "ro", markersize=markersize)

    plt.savefig("../figures/poster/mtt_pipeline.pdf", bbox_inches="tight")
    plt.show()
    plt.close()

In [ ]:
mean_energy = np.mean(
    [pos_img[1].cpu().clamp(min=0).sum(dim=[-1, -2]) for _, pos_img, _ in stacked]
)
mean_cardinality = np.mean(
    [len(info["target_positions"]) for _, _, infos in stacked for info in infos]
)
mean_energy, mean_cardinality

In [ ]:
card_true = np.array([len(d[2][filt_idx]["target_positions"]) for d in stacked])
card_cnn = np.array([len(p) for p in predictions_cnn])
card_img = pred_img.sum(axis=(-1, -2))

In [ ]:
# plot cardinality
plt.step(np.arange(n), card_true, label="Ground Truth")
plt.step(np.arange(n), card_cnn, label="Extracted", linestyle="dashed")
plt.step(np.arange(n), card_img, label="CNN Output Sum", linestyle="dashed")
plt.xlabel("Time (s)")
plt.ylabel("# of targets")
# legend to the right of axis
plt.legend(bbox_to_anchor=(0.5, 1.05), loc="lower center", ncol=2)
plt.show()

In [ ]:
ospa_cnn = []
ospa_cnn_components = []
for idx in range(n):
    sensor_img, target_img, info = stacked[idx]
    target_positions = info[-1]["target_positions"]
    ospa_cnn += [compute_ospa(target_positions, predictions_cnn[idx], 500, p=2)]
    ospa_cnn_components += [
        compute_ospa_components(target_positions, predictions_cnn[idx], 500, p=2)
    ]

ospa_cnn_components = np.array(ospa_cnn_components)
print(f"Mean OSPA: {np.mean(ospa_cnn):.2f}m \u00B1 {np.std(ospa_cnn):.2f}m")
print(
    f"Mean OSPA (Position): {np.mean(ospa_cnn_components[:,0]):.2f}m \u00B1 {np.std(ospa_cnn_components[:,0]):.2f}m"
)
print(
    f"Mean OSPA (Cardinality): {np.mean(ospa_cnn_components[:,1]):.2f}m \u00B1 {np.std(ospa_cnn_components[:,1]):.2f}m"
)

plt.figure()
plt.step(np.arange(n), ospa_cnn, label="CNN")
# plt.step(np.arange(n), ospa_cnn_components[:,0], label="CNN (Position)")
# plt.step(np.arange(n), ospa_cnn_components[:,1], label="CNN (Cardinality)")
plt.ylabel("OSPA (m)")
plt.xlabel("Time (s)")
plt.legend()

## Make a figure of the CNN results at different scales

In [ ]:
images = []
scales = list(range(1, 6))

for scale in scales:
    with open(f"../data/test/{scale}km/simulations.pkl", "rb") as f:
        dataset_vectors: List[List[VectorData]] = pkl.load(f)
    stacked = simulation_window(
        stack_images(
            map(
                partial(vector_to_image, img_size=128 * scale, device="cuda"),
                dataset_vectors[rng.integers(100)][:20],
            )
        ),
        idx="random",
    )[0]
    sensor_img = stacked.sensor_images[filt_idx].cpu().numpy()
    target_img = stacked.target_images[filt_idx].cpu().numpy()
    with torch.no_grad():
        cnn_img = model(stacked.sensor_images.cuda()).squeeze().cpu().numpy()

    images += [(sensor_img, target_img, cnn_img)]

In [ ]:
with sns.axes_style("ticks"):
    fig, axs = plt.subplots(2, len(images), figsize=(7.5, 3.0))
for scale, (sensor_img, target_img, cnn_img) in zip(scales, images):
    ax_sensor, ax_cnn = axs[:, scale - 1]

    cmap = "gray_r"

    ax_sensor.imshow(np.log(sensor_img + 1e-3), cmap=cmap)
    ax_sensor.set_title(f"{scale}km wide area")
    ax_cnn.imshow(cnn_img, cmap=cmap)

    for ax in (ax_sensor, ax_cnn):
        ax.set_xticks([])
        ax.set_yticks([])
    if scale == scales[0]:
        ax_sensor.set_ylabel("Sensor Image")
        ax_cnn.set_ylabel("CNN Output")
plt.savefig("../figures/mtt_transfer.pdf")
plt.savefig("../figures/mtt_transfer.png")

In [ ]:
with sns.axes_style("ticks"), sns.plotting_context("poster"):
    fig, axs = plt.subplots(2, len(images), figsize=(15.0, 6.0))
    for scale, (sensor_img, target_img, cnn_img) in zip(scales, images):
        ax_sensor, ax_cnn = axs[:, scale - 1]

        cmap = "gray_r"

        ax_sensor.imshow(np.log(sensor_img + 1e-3), cmap=cmap)
        ax_sensor.set_title(f"{scale}km wide area")
        ax_cnn.imshow(cnn_img, cmap=cmap)

        for ax in (ax_sensor, ax_cnn):
            ax.set_xticks([])
            ax.set_yticks([])
        if scale == scales[0]:
            ax_sensor.set_ylabel("Sensor Image")
            ax_cnn.set_ylabel("CNN Output")
plt.savefig("../figures/poster/mtt_transfer.pdf")
plt.savefig("../figures/poster/mtt_transfer.png")